In [1]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [2]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit, substring, avg, sum

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/gold/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-79ea25ff-a94b-49c0-9edc-b68aa54a8c0c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 242ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      

In [3]:
# apagar tudo do hdfs

#s_path = "/projeto/fisico/gold/"
b_path = "/projeto/fisico/gold/physical/"

#client.delete(s_path,True)
client.delete(b_path,True)

False

In [4]:
# read data from the silver tables
s_df = spark.table("silver.stats")
b_df = spark.table("silver.bios")

In [5]:
s_df.printSchema()
b_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Team: double (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: double (nullable = true)
 |-- TSP: double (nullable = true)
 |-- 3PAr: double (nullable = true)
 |-- FTr: double (nullable = true)
 |-- ORBP: double (nullable = true)
 |-- DRBP: double (nullable = true)
 |-- TRBP: double (nullable = true)
 |-- ASTP: double (nullable = true)
 |-- STLP: double (nullable = true)
 |-- BLKP: double (nullable = true)
 |-- TOVP: double (nullable = true)
 |-- USGP: double (nullable = true)
 |-- OWS: double (nullable = true)
 |-- DWS: double (nullable = true)
 |-- WS: double (nullable = true)
 |-- WS48: double (nullable = true)
 |-- OBPM: double (nullable = true)
 |-- DBPM: double (nullable = true)
 |-- BPM: double (nullable = true)


In [6]:
# join to make a flat table
gd_df = s_df \
    .join(b_df, s_df.Player == b_df.name) \
    .select("Player", "height_cm", "weight_kg", "imc")

In [7]:
gd_df.printSchema()
gd_df.toPandas()

root
 |-- Player: string (nullable = true)
 |-- height_cm: double (nullable = true)
 |-- weight_kg: double (nullable = true)
 |-- imc: double (nullable = true)



Player  height_cm   weight_kg        imc
0            Quincy Acy     200.66  108.862169  27.034129
1          Jordan Adams     195.58   94.800805  24.781076
2          Steven Adams     213.36  120.201978  26.402353
3          Steven Adams     213.36  120.201978  26.402353
4          Steven Adams     213.36  115.666054  25.406037
...                 ...        ...         ...        ...
27971     Jack Stephens     190.50   83.914588  23.120889
27972        Jim Tucker     200.66   83.914588  20.838808
27973  Ernie Vandeweghe     190.50   88.450512  24.370667
27974      Bob Williams     198.12  104.326245  26.576266
27975     Max Zaslofsky     187.96   77.110703  21.824324

[27976 rows x 4 columns]

In [8]:
# aggregate data per country, year and product (name and color)
gd_df = gd_df \
    .groupBy("Player") \
    .agg(
        avg(gd_df.height_cm).alias("height_cm"),
        avg(gd_df.weight_kg).alias("weight_kg"),
        avg(gd_df.imc).alias("imc"),
    )

In [9]:
import pyspark.sql.functions as func
gd_df = gd_df \
    .withColumn("height_cm", func.round(gd_df["height_cm"], 2)) \
    .withColumn("weight_kg", func.round(gd_df["weight_kg"], 2)) \
    .withColumn("imc", func.round(gd_df["imc"], 2))
gd_df.show()

+----------------+---------+---------+-----+
|          Player|height_cm|weight_kg|  imc|
+----------------+---------+---------+-----+
| Al-Farouq Aminu|   205.74|    99.79|23.57|
|   Greg Stiemsma|   210.82|   117.93|26.53|
|  Rodney Stuckey|   195.58|    95.25| 24.9|
|  Brandon Ingram|   205.74|    86.18|20.36|
|     Terry Mills|   208.28|   104.33|24.05|
|  Nate Blackwell|   193.04|    77.11|20.69|
|     Zach LaVine|   195.58|    91.47|23.91|
|    Andre Miller|    190.5|    90.72| 25.0|
|  Kevin Seraphin|   205.74|   129.27|30.54|
|  Viktor Khryapa|   205.74|    95.25| 22.5|
|Shandon Anderson|   198.12|    94.35|24.03|
|  Marcus Haislip|   208.28|   104.33|24.05|
|      Lee Nailon|   205.74|   107.95| 25.5|
|   Eric Williams|    203.2|    99.79|24.17|
|  Terry Cummings|   205.74|    99.79|23.57|
|   Lloyd Daniels|   200.66|    92.99|23.09|
|    Carl Herrera|   205.74|    97.52|23.04|
|    Bobby Phills|   195.58|    95.25| 24.9|
|   Michael Smith|   205.74|   103.19|24.39|
|     Aaro

In [10]:
# write to delta table
gd_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/gold/physical/")

22/01/15 15:09:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
# check the results in the table
spark.table("gold.physical").show()
spark.table("gold.physical").count()

+----------------+----+---------+---------+-----+
|          Player| Age|height_cm|weight_kg|  imc|
+----------------+----+---------+---------+-----+
| Al-Farouq Aminu|null|   205.74|    99.79|23.57|
|   Greg Stiemsma|null|   210.82|   117.93|26.53|
|  Rodney Stuckey|null|   195.58|    95.25| 24.9|
|  Brandon Ingram|null|   205.74|    86.18|20.36|
|     Terry Mills|null|   208.28|   104.33|24.05|
|  Nate Blackwell|null|   193.04|    77.11|20.69|
|     Zach LaVine|null|   195.58|    91.47|23.91|
|    Andre Miller|null|    190.5|    90.72| 25.0|
|  Kevin Seraphin|null|   205.74|   129.27|30.54|
|  Viktor Khryapa|null|   205.74|    95.25| 22.5|
|Shandon Anderson|null|   198.12|    94.35|24.03|
|  Marcus Haislip|null|   208.28|   104.33|24.05|
|      Lee Nailon|null|   205.74|   107.95| 25.5|
|   Eric Williams|null|    203.2|    99.79|24.17|
|  Terry Cummings|null|   205.74|    99.79|23.57|
|   Lloyd Daniels|null|   200.66|    92.99|23.09|
|    Carl Herrera|null|   205.74|    97.52|23.04|


3892